In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.imagenet_utils import preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexPE"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = VGG16(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 512)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 512)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.8min


[CV] ........................... C=0.01, score=0.883858, total= 1.9min
[CV] C=1.0 ...........................................................
[CV] ............................ C=1.0, score=0.886811, total= 1.8min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.908203, total= 1.9min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.910784, total= 1.9min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.917659, total= 1.9min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.914767, total= 1.9min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.918945, total= 1.9min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.0min


[CV] ............................ C=0.1, score=0.918731, total= 2.0min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.927291, total= 2.0min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.912916, total= 2.0min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.893701, total= 2.0min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.915354, total= 2.0min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.916016, total= 2.1min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.913725, total= 2.1min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.917808, total= 1.4min
[CV] ................... C=1000000000.0, score=0.888780, total= 1.4min
[CV] ................... C=1000000000.0, score=0.907843, total= 1.4min
[CV] ................... C=1000000000.0, score=0.870425, total= 1.2min
[CV] ................... C=1000000000.0, score=0.898622, total= 1.3min
[CV] ................... C=1000000000.0, score=0.897614, total= 1.2min
[CV] ................... C=1000000000.0, score=0.897919, total= 1.3min
[CV] ................... C=1000000000.0, score=0.899802, total= 1.2min
[CV] ................... C=1000000000.0, score=0.918327, total= 1.2min
[CV] .................. C=10000000000.0, score=0.884766, total=  54.2s
[CV] .................. C=10000000000.0, score=0.909002, total=  48.9s
[CV] .................. C=10000000000.0, score=0.890196, total=  47.5s
[CV] .................. C=10000000000.0, score=0.907830, total=  39.5s
[CV] .................. C=10000000000.0, score=0.910537, total=  38.4s
[CV] .

[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 10.6min remaining:   25.4s


[CV] .................. C=10000000000.0, score=0.892186, total=  42.5s
[CV] .................. C=10000000000.0, score=0.909449, total=  44.5s
[CV] .................. C=10000000000.0, score=0.921315, total=  39.0s
[CV] .................. C=10000000000.0, score=0.899802, total=  42.6s


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 10.6min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([112.54467797, 120.5491307 , 127.26999075, 129.57217674,
        133.20848944, 128.14692254, 134.27709002, 126.51583157,
        133.35773447, 123.44512622,  89.44724815,  76.63637807,
         44.13734016]),
 'mean_score_time': array([0.03369195, 0.02564082, 0.02826698, 0.02727599, 0.01996422,
        0.01948166, 0.02063787, 0.02228429, 0.01931975, 0.01712689,
        0.00956693, 0.00714731, 0.00565569]),
 'mean_test_score': array([0.90726125, 0.91436464, 0.90568272, 0.90341358, 0.90282163,
        0.89996054, 0.90134175, 0.90124309, 0.90459747, 0.90104578,
        0.89996054, 0.90065114, 0.89877664]),
 'mean_train_score': array([0.94311791, 0.96886813, 0.96347171, 0.96427575, 0.96264529,
        0.95902477, 0.96320259, 0.96182147, 0.96377091, 0.96363021,
        0.95938711, 0.96118793, 0.95845799]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.90726, std: 0.01191, params: {'C': 0.01},
 mean: 0.91436, std: 0.01008, params: {'C': 0.1},
 mean: 0.90568, std: 0.01480, params: {'C': 1.0},
 mean: 0.90341, std: 0.01374, params: {'C': 10.0},
 mean: 0.90282, std: 0.00659, params: {'C': 100.0},
 mean: 0.89996, std: 0.01871, params: {'C': 1000.0},
 mean: 0.90134, std: 0.01024, params: {'C': 10000.0},
 mean: 0.90124, std: 0.01168, params: {'C': 100000.0},
 mean: 0.90460, std: 0.01271, params: {'C': 1000000.0},
 mean: 0.90105, std: 0.01306, params: {'C': 10000000.0},
 mean: 0.89996, std: 0.01208, params: {'C': 100000000.0},
 mean: 0.90065, std: 0.01344, params: {'C': 1000000000.0},
 mean: 0.89878, std: 0.01590, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.1} - score: 0.9144
